In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sqlalchemy import create_engine
import mysql.connector
import pickle

%matplotlib inline

import getpass  # To get the password without showing the input
password = getpass.getpass()

········


In [2]:
# Connect to MySQL database
conn = mysql.connector.connect(host='127.0.0.1', user='root', passwd=password)
cursor = conn.cursor()

#Select the database
cursor.execute("USE vanguard")

In [3]:
# how far people got in process

cursor.execute("""
                    SELECT clients.client_id,
                        clients.clnt_age, 
                        clients.gendr, 
                        COUNT(distinct(interactions.process_step)) AS count_of_steps,
                        group_id.variation
                    FROM clients
                    JOIN interactions
                    ON clients.client_id = interactions.client_id
                    INNER JOIN group_id
                    ON group_id.client_id = clients.client_id
                    GROUP BY clients.client_id, clients.clnt_age, clients.gendr,group_id.variation;

                    """)
results = cursor.fetchall()

how_far_in_process = pd.DataFrame(results)

In [4]:
# how much time spent per step

cursor.execute("""
                    WITH time AS (
    SELECT 
        visit_id,
        interactions.client_id,
        process_step,
        date_time,
        group_id.variation AS variation,
        LEAD(date_time) OVER (PARTITION BY visit_id, client_id ORDER BY date_time) AS next_step_time
    FROM interactions
    JOIN group_id
    ON group_id.client_id = interactions.client_id
)
SELECT 
    visit_id,
    client_id,
    process_step,
    date_time,
    next_step_time,
    TIMESTAMPDIFF(SECOND, date_time, next_step_time) AS time_difference_seconds,
    variation
FROM time
GROUP BY visit_id,
    client_id,
    process_step, 
    date_time,
    next_step_time,
    time_difference_seconds,
    variation 
ORDER BY client_id, 
		visit_id, 
        date_time,
        process_step 
        ;

                    """)
results = cursor.fetchall()

how_long_in_step = pd.DataFrame(results)

In [5]:
# how many times on a same step (error)

cursor.execute("""
                    SELECT 
    visit_id,
    client_id,
    process_step,
    COUNT(*) AS step_count
FROM interactions
GROUP BY 
    visit_id,
    client_id,
    process_step
ORDER BY 
    client_id,
    visit_id,
    process_step;

                    """)
results = cursor.fetchall()

error_step = pd.DataFrame(results)

In [6]:
# all data for control group

cursor.execute("""
                    SELECT *
FROM clients
JOIN interactions
ON clients.client_id = interactions.client_id
JOIN group_id
ON group_id.client_id = interactions.client_id
WHERE group_id.variation = 'Control';

                    """)
results = cursor.fetchall()

control_group = pd.DataFrame(results)

In [7]:
# all data for test group

cursor.execute("""
                    SELECT *
FROM clients
JOIN interactions
ON clients.client_id = interactions.client_id
JOIN group_id
ON group_id.client_id = interactions.client_id
WHERE group_id.variation = 'Test';

                    """)
results = cursor.fetchall()

test_group = pd.DataFrame(results)

In [8]:
# test clients who made it to confirm

cursor.execute("""
                    SELECT clients.client_id, COUNT(process_step)
FROM clients
JOIN interactions
ON clients.client_id = interactions.client_id
JOIN group_id
ON group_id.client_id = interactions.client_id
WHERE interactions.process_step = '4' AND group_id.variation = 'Test'
GROUP BY clients.client_id;

                    """)
results = cursor.fetchall()

test_confirmed = pd.DataFrame(results)

In [9]:
# test clients who dnf

cursor.execute("""
                    SELECT clients.client_id, COUNT(process_step)
FROM clients
JOIN interactions
ON clients.client_id = interactions.client_id
JOIN group_id
ON group_id.client_id = interactions.client_id
WHERE group_id.variation = 'Test' AND group_id.client_id NOT IN (SELECT clients.client_id
FROM clients
JOIN interactions
ON clients.client_id = interactions.client_id
JOIN group_id
ON group_id.client_id = interactions.client_id
WHERE interactions.process_step = '4' AND group_id.variation = 'Test')
GROUP BY clients.client_id;

                    """)
results = cursor.fetchall()

test_dnf = pd.DataFrame(results)

In [10]:
# control clients who made it to confirm

cursor.execute("""
                    SELECT clients.client_id,  COUNT(process_step)
FROM clients
JOIN interactions
ON clients.client_id = interactions.client_id
JOIN group_id
ON group_id.client_id = interactions.client_id
WHERE interactions.process_step = '4' AND group_id.variation = 'Control'
GROUP BY clients.client_id;

                    """)
results = cursor.fetchall()

control_confirmed = pd.DataFrame(results)

In [11]:
# control clients who dnf

cursor.execute("""
                    SELECT clients.client_id, COUNT(process_step)
FROM clients
JOIN interactions
ON clients.client_id = interactions.client_id
JOIN group_id
ON group_id.client_id = interactions.client_id
WHERE group_id.variation = 'Control' AND clients.client_id NOT IN (SELECT clients.client_id
FROM clients
JOIN interactions
ON clients.client_id = interactions.client_id
JOIN group_id
ON group_id.client_id = interactions.client_id
WHERE interactions.process_step = '4' AND group_id.variation = 'Control')
GROUP BY clients.client_id;

                    """)
results = cursor.fetchall()

control_dnf = pd.DataFrame(results)

In [12]:
#pickles!

import pickle

def pickle_object(obj, filename):
    """
    Pickle an object to a file.
    
    Args:
        obj: The object to pickle.
        filename (str): The filename to save the pickled object.
        
    Returns:
        None
    """
    with open(filename, 'wb') as f:
        pickle.dump(obj, f)

In [13]:
pickle_object(control_dnf, 'control_dnf.pkl')

In [14]:
pickle_object(control_confirmed, 'control_confirmed.pkl')
pickle_object(test_dnf, 'test_dnf.pkl')
pickle_object(test_confirmed, 'test_confirmed.pkl')
pickle_object(test_group, 'test_group.pkl')
pickle_object(control_group, 'control_group.pkl')
pickle_object(error_step, 'error_step.pkl')
pickle_object(how_long_in_step, 'how_long_in_step.pkl')
pickle_object(how_far_in_process, 'how_far_in_process.pkl')

In [15]:

def unpickle_object(filename):
    """
    Unpickle an object from a file.
    
    Args:
        filename (str): The filename of the pickled object.
        
    Returns:
        The unpickled object.
    """
    with open(filename, 'rb') as f:
        obj = pickle.load(f)
    return obj

# Example usage:
data = unpickle_object('control_dnf.pkl')
print(data)

            0  1
0     9294070  2
1     2842481  4
2     1218661  2
3      800925  4
4     1242238  1
...       ... ..
8093  5429189  4
8094  8499329  1
8095  4870070  6
8096  8249112  3
8097  1677096  7

[8098 rows x 2 columns]
